# Imports 

In [2]:
import pandas as pd

# Load data 

In [3]:
transactions = pd.read_csv('../data/transactions.csv')
accounts = pd.read_csv('../data/accounts.csv')
users = pd.read_csv('../data/users.csv')

# Explore transactions

In [4]:
transactions.head()

,date,amount,account_id
0,2019-12-16,-29.09,1
1,2019-12-16,160.00,1
2,2019-12-16,-14.00,1
3,2019-12-17,-94.50,1
4,2019-12-17,50.00,1


In [14]:
transactions.shape

(191398, 3)

In [9]:
transactions.describe()

,amount,account_id
count,191398.000000,191398.000000
mean,3.902326,224.494054
std,1202.288020,127.191401
min,-82201.680000,0.000000
25%,-53.700000,110.000000
50%,-15.900000,226.000000
75%,-2.000000,337.000000
max,83340.000000,440.000000


In [11]:
transactions.dtypes

date           object
amount        float64
account_id      int64
dtype: object

In [12]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191398 entries, 0 to 191397
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        191398 non-null  object 
 1   amount      191398 non-null  float64
 2   account_id  191398 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 4.4+ MB


In [24]:
transactions.duplicated().value_counts()

False    181542
True       9856
dtype: int64

# Explore accounts 

In [8]:
accounts.head()

,id,balance,user_id
0,0,3000.00,112
1,1,-465.66,112
2,2,372.81,104
3,3,229.73,198
4,4,1998.48,316


In [15]:
accounts.shape

(441, 3)

In [19]:
accounts.describe()

,id,balance,user_id
count,441.00000,441.000000,441.000000
mean,220.00000,1978.075986,172.231293
std,127.44999,9892.338349,99.099541
min,0.00000,-13474.330000,0.000000
25%,110.00000,0.480000,85.000000
50%,220.00000,59.200000,172.000000
75%,330.00000,1169.360000,258.000000
max,440.00000,154251.970000,342.000000


In [20]:
accounts.dtypes

id           int64
balance    float64
user_id      int64
dtype: object

In [21]:
accounts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441 entries, 0 to 440
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       441 non-null    int64  
 1   balance  441 non-null    float64
 2   user_id  441 non-null    int64  
dtypes: float64(1), int64(2)
memory usage: 10.5 KB


In [25]:
accounts.duplicated().value_counts()

False    441
dtype: int64

# Explore users

In [7]:
users.head()

,id;update_date
0,0;27/09/2020
1,1;27/09/2020
2,2;27/09/2020
3,3;27/09/2020
4,4;27/09/2020


In [27]:
users.shape

(343, 1)

In [29]:
users.describe()

,id;update_date
count,343
unique,343
top,0;27/09/2020
freq,1


In [31]:
users.dtypes

id;update_date    object
dtype: object

In [33]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id;update_date  343 non-null    object
dtypes: object(1)
memory usage: 2.8+ KB


In [34]:
users.duplicated().value_counts()

False    343
dtype: int64

# Join datasets